# فہرست مضامین
- [گائیڈنس کے بارے میں](../../../../code/01.Introduce)
- [سیٹ اپ](../../../../code/01.Introduce)
- [بے قید تخلیق](../../../../code/01.Introduce)
- [فی 3 کے لیے بات کرنا](../../../../code/01.Introduce)
- [ریگیکس](../../../../code/01.Introduce)
- [انتخاب](../../../../code/01.Introduce)
- [خیالات کی زنجیر](../../../../code/01.Introduce)
- [JSON تخلیق](../../../../code/01.Introduce)
- [HTML تخلیق](../../../../code/01.Introduce)


# گائیڈنس کے بارے میں  
گائیڈنس ایک آزمودہ اوپن سورس پائتھون لائبریری ہے جو کسی بھی زبان کے ماڈل (LM) کے نتائج کو کنٹرول کرنے کے لیے استعمال ہوتی ہے۔ ایک API کال کے ذریعے، آپ (پائتھون میں) وہ مخصوص پروگراماتی پابندیاں بیان کر سکتے ہیں جن پر ماڈل کو عمل کرنا ہوگا اور مطلوبہ ساخت میں آؤٹ پٹ تیار کر سکتے ہیں، جیسے JSON، پائتھون، HTML، SQL، یا کوئی بھی ساخت جو آپ کے استعمال کے کیس کے لیے ضروری ہو۔

گائیڈنس روایتی پرامپٹنگ تکنیکوں سے مختلف ہے۔ یہ ماڈل کو انفرنس لیئر میں ٹوکن بہ ٹوکن ہدایت دے کر پابندیاں نافذ کرتا ہے، جس سے اعلیٰ معیار کے نتائج حاصل ہوتے ہیں اور سخت ساخت والے منظرناموں میں استعمال کرتے وقت لاگت اور تاخیر کو 30–50% تک کم کیا جا سکتا ہے۔

گائیڈنس کے بارے میں مزید جاننے کے لیے، [GitHub پر عوامی ریپوزٹری](https://github.com/guidance-ai/guidance) دیکھیں یا Microsoft Build میں [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) دیکھیں۔


# سیٹ اپ
1. Guidance کو `pip install guidance --pre` کے ذریعے انسٹال کریں
2. Azure میں Phi 3.5 mini endpoint کو https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml پر جا کر اور "Deploy" بٹن پر کلک کر کے ڈیپلوئے کریں
3. اپنے endpoint کی API key کو `AZURE_PHI3_KEY` نامی ایک ماحول کے متغیر میں محفوظ کریں اور URL کو `AZURE_PHI3_URL` نامی ایک ماحول کے متغیر میں محفوظ کریں


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# غیر محدود تخلیق
متن کو بغیر کسی پابندی کے `gen()` فنکشن کے ذریعے تخلیق کیا جا سکتا ہے۔ یہ بالکل ایسا ہی ہے جیسے ماڈل کو Guidance کے بغیر استعمال کرنا۔

## چیٹ فارمیٹنگ
بہت سے چیٹ ماڈلز کی طرح، Phi-3 صارف اور معاون کے درمیان پیغامات کو ایک مخصوص فارمیٹ میں توقع کرتا ہے۔ Guidance Phi-3 کے چیٹ ٹیمپلیٹ کو سپورٹ کرتا ہے اور آپ کے لیے چیٹ فارمیٹنگ کو منظم کرے گا۔ چیٹ کے حصے بنانے کے لیے، گفتگو کے ہر حصے کو `with user()` یا `with assistant()` بلاک میں رکھیں۔ ایک `with system()` بلاک کو سسٹم پیغام سیٹ کرنے کے لیے استعمال کیا جا سکتا ہے۔


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## ٹوکن کی بچت
بہت منظم حالات میں، Guidance ٹوکنز کو چھوڑ سکتا ہے اور صرف ضروری ٹوکنز پیدا کر سکتا ہے، جس سے کارکردگی بہتر ہوتی ہے، کارکردگی میں اضافہ ہوتا ہے اور API کے اخراجات بچتے ہیں۔ پیدا کیے گئے ٹوکنز اس نوٹ بک میں نمایاں پس منظر کے ساتھ دکھائے گئے ہیں۔ جبری ٹوکنز بغیر نمایاں کیے دکھائے جاتے ہیں اور ان کی قیمت ان پٹ ٹوکنز کے برابر ہوتی ہے، جو آؤٹ پٹ ٹوکنز کی قیمت کا تقریباً ایک تہائی ہوتی ہے۔

*نوٹ:* پہلے مثال میں بغیر کسی پابندی کے جنریشن ممکن نہیں تھی کہ کوئی ٹوکنز کو مجبور کیا جا سکے کیونکہ ہم نے کوئی پابندیاں فراہم نہیں کی تھیں۔


# فی 3 کے لیے بات کرنا  
رہنمائی کے ذریعے، آپ آسانی سے ماڈل کے جوابات میں متن شامل کر سکتے ہیں۔ یہ اس وقت مددگار ثابت ہو سکتا ہے جب آپ ماڈل کے نتائج کو کسی خاص سمت میں لے جانا چاہتے ہوں۔


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# ریگیکس کے ساتھ پابندی لگانا  
پچھلی مثال میں، Phi 3 نے `Canberra` کے جواب دینے کے بعد مزید وضاحتیں فراہم کیں۔ ماڈل کے آؤٹ پٹ کو صرف ایک لفظ تک محدود کرنے کے لیے، ریگیکس استعمال کیا جا سکتا ہے۔  


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

ریجیکس کے ساتھ، صرف لفظ `کینبرا` پیدا ہوتا ہے۔


# متعدد انتخابوں میں سے انتخاب کرنا
جب کچھ ممکنہ انتخاب معلوم ہوں، تو آپ `select()` فنکشن استعمال کر سکتے ہیں تاکہ ماڈل کو اختیارات کی فہرست میں سے انتخاب کرنے دیں۔


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`کے ساتھ select()، صرف token "Can" پیدا ہوا۔ چونکہ "Canberra" واحد آپشن ہے جو ممکنہ طور پر جواب مکمل کر سکتا ہے، باقی tokens کو مجبور کیا گیا۔`


# سوچ کی زنجیر
سوچ کی زنجیر ایک تکنیک ہے جو ماڈل کے نتائج کے معیار کو بہتر بنانے میں مدد دے سکتی ہے، اسے مسئلے کو قدم بہ قدم حل کرنے کی ترغیب دے کر۔ عام طور پر، حتمی جواب تک پہنچنے کے لیے متعدد پرامپٹ ٹرنز کی ضرورت ہوتی ہے۔ پہلے، ماڈل کو ہدایت دیں کہ وہ قدم بہ قدم سوچے۔ پھر، ماڈل کو دوبارہ پرامپٹ کریں تاکہ وہ حتمی جواب فراہم کرے۔ معیاری چیٹ انفرنس APIs کے ساتھ، اس میں 2 API کالز لگتی ہیں، اور ماڈل کی تیار کردہ "سوچ کی زنجیر" دو بار چارج ہوتی ہے – ایک بار جب ماڈل نے اسے آؤٹ پٹ ٹوکنز کے طور پر تیار کیا، اور پھر دوسری کال کے لیے ان پٹ ٹوکنز کے طور پر۔ Guidance کے ساتھ، پورا کثیر مرحلہ عمل ایک ہی API کال کے حصے کے طور پر پروسیس اور چارج ہوتا ہے، جس سے لاگت اور تاخیر کم ہوتی ہے۔


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON جنریشن
گائیڈنس کو استعمال کیا جا سکتا ہے تاکہ JSON کو ایک JSON اسکیمہ یا پائیڈینٹک ماڈل کے مطابق بنایا جا سکے، جیسے کہ یہاں دکھائی گئی یوزر پروفائل اسکیمہ۔


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## ایچ ٹی ایم ایل جنریشن

گائیڈنس کوڈ بنانے اور پروگرامنگ زبان کے نحوی تقاضوں کو پورا کرنے کے لیے بھی استعمال کیا جا سکتا ہے۔ اس سیکشن میں، ہم ایک چھوٹا گائیڈنس پروگرام بنائیں گے جو بہت سادہ ایچ ٹی ایم ایل ویب صفحات لکھنے کے لیے استعمال ہوگا۔

ہم ویب صفحہ کو چھوٹے حصوں میں تقسیم کریں گے، ہر ایک کا اپنا گائیڈنس فنکشن ہوگا۔ پھر ان کو ہمارے آخری فنکشن میں ملا کر ایک ایچ ٹی ایم ایل ویب صفحہ بنائیں گے۔
اس کے بعد ہم اس فنکشن کو Azure AI میں گائیڈنس-فعال ماڈل کے خلاف چلائیں گے۔

*نوٹ:* یہ ایک مکمل خصوصیات والا ایچ ٹی ایم ایل جنریٹر نہیں ہوگا؛ مقصد یہ دکھانا ہے کہ آپ اپنی انفرادی ضروریات کے لیے ساختی آؤٹ پٹ کیسے بنا سکتے ہیں۔

ہم گائیڈنس سے درکار چیزیں درآمد کرنے سے شروع کرتے ہیں:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML ویب صفحات بہت منظم ہوتے ہیں، اور ہم ان صفحات کے حصوں کو Guidance کے ذریعے 'مجبور' کریں گے۔  
جب ہم ماڈل سے واضح طور پر متن طلب کرتے ہیں، تو ہمیں یہ یقینی بنانا ہوگا کہ اس میں کوئی ایسی چیز شامل نہ ہو جو ٹیگ ہو سکتی ہے - یعنی ہمیں '<' اور '>' کرداروں کو خارج کرنا ہوگا۔


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

ہم پھر اس فنکشن کو استعمال کر سکتے ہیں تاکہ کسی بھی من چاہے HTML ٹیگ کے اندر متن تیار کریں:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

اب، آئیے صفحے کا ہیڈر بنائیں۔  
اس کے حصے کے طور پر، ہمیں صفحے کا عنوان تیار کرنا ہوگا:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML صفحے کے مواد میں سرخیاں اور پیراگراف شامل ہوں گے۔  
ہم ہر ایک کے لیے ایک فنکشن تعریف کر سکتے ہیں:  


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

اب، HTML کے جسم کو خود تعریف کرنے کے لیے فنکشن۔

یہ `select()` کو `recurse=True` کے ساتھ استعمال کرتا ہے تاکہ متعدد سرخیاں اور پیراگراف تیار کیے جا سکیں:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

اب ہم اس فنکشن پر آتے ہیں جو مکمل HTML صفحہ تیار کرتا ہے۔  
ہم HTML کا آغاز کرنے والا ٹیگ شامل کرتے ہیں، پھر ہیڈر تیار کرتے ہیں، پھر باڈی، اور آخر میں HTML کا اختتامی ٹیگ شامل کرتے ہیں۔  


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

ہم ایک صارف دوست لفافہ فراہم کرتے ہیں، جو ہمیں اجازت دے گا کہ ہم:
- جنریشن کا درجہ حرارت مقرر کریں
- ماڈل آبجیکٹ سے تیار کردہ صفحہ حاصل کریں


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

ہم پھر آؤٹ پٹ کو ایک فائل میں لکھ سکتے ہیں:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

اور [نتیجہ دیکھیں](../../../../code/01.Introduce/sample_page.html)۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
